In [3]:
import tarfile
import sqlite3
import csv
import pandas as pd


In [36]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName('Large tar.gz Processing').getOrCreate()

# Read data directly using Spark's capabilities
df = spark.read.csv('20181113_training_set.tar.gz')
df.show(10)

# Print the list of columns
print(df.columns)

# Stop the Spark session
spark.stop()

+--------------------+----------------+--------------+--------------------+------+------+------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+-------+---------------+--------------------+--------------------+
|                 _c0|             _c1|           _c2|                 _c3|   _c4|   _c5|   _c6|        _c7|           _c8|                 _c9|                _c10|                _c11|                _c12|                _c13|                _c14|       _c15|      _c16|   _c17|           _c18|                _c19|                _c20|
+--------------------+----------------+--------------+--------------------+------+------+------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+-------+---------------+--------------------+-----

In [29]:
!ls extracted/training_set/


log_0_20180815_000000000000.csv log_4_20180816_000000000000.csv
log_2_20180908_000000000000.csv log_5_20180809_000000000000.csv
log_3_20180829_000000000000.csv log_9_20180820_000000000000.csv
log_4_20180719_000000000000.csv


In [4]:
df = pd.read_csv(filepath_or_buffer="training_set/log_0_20180716_000000000000.csv")
df.head()

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,1_00001238-3f7a-4fd2-b52e-af4237a41ed9,1,20,t_91eb6576-0f1f-4394-92d8-560d7b8f632b,True,True,True,False,0,0,...,0,0,0,True,5,2018-07-16,True,editorial_playlist,fwdbtn,fwdbtn
1,1_00001238-3f7a-4fd2-b52e-af4237a41ed9,2,20,t_338cb0f3-0035-418f-ab2d-3f9342815aaa,True,True,True,False,0,1,...,0,0,0,True,5,2018-07-16,True,editorial_playlist,fwdbtn,fwdbtn
2,1_00001238-3f7a-4fd2-b52e-af4237a41ed9,3,20,t_98e73634-8f33-4275-b544-3451553f431e,False,True,True,False,0,1,...,0,0,0,True,5,2018-07-16,True,editorial_playlist,fwdbtn,endplay
3,1_00001238-3f7a-4fd2-b52e-af4237a41ed9,4,20,t_2fd6279a-06cd-4d5d-9e54-1912f51bb3c3,False,False,False,True,0,1,...,0,0,0,False,6,2018-07-16,True,editorial_playlist,clickrow,trackdone
4,1_00001238-3f7a-4fd2-b52e-af4237a41ed9,5,20,t_de860a61-bb57-4386-9d5a-6ba111c4c266,False,False,False,True,0,1,...,0,0,0,False,6,2018-07-16,True,editorial_playlist,trackdone,trackdone


In [5]:
df.columns

Index(['session_id', 'session_position', 'session_length', 'track_id_clean',
       'skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch',
       'no_pause_before_play', 'short_pause_before_play',
       'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
       'hist_user_behavior_n_seekback', 'hist_user_behavior_is_shuffle',
       'hour_of_day', 'date', 'premium', 'context_type',
       'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end'],
      dtype='object')

In [49]:
len(df)

3392315

In [50]:
df["session_id"].nunique()

203225

In [6]:
track_df = pd.read_csv("track_features/tf_000000000000.csv")
track_df.head()

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,t_2e8f4b71-8a0b-4b9c-b7d8-fb5208e87f9f,326.013336,1971,99.582885,0.716209,0.366495,0.332605,0.439835,5.805774,0.238847,...,4,0.223395,0.146012,-0.706908,0.259496,0.481157,0.238427,-0.098389,-0.254960,-0.227383
1,t_dae2ec0e-ec7b-4b3e-b60c-4a884d0eccb0,147.813324,1963,97.272035,0.839460,0.362212,0.389829,0.507580,6.845427,0.420476,...,4,0.484702,0.039554,-0.539554,0.105141,0.692589,0.226047,-0.468162,0.164389,-0.769024
2,t_cf0164dd-1531-4399-bfa6-dec19cd1fedc,110.400002,1974,99.620384,0.054673,0.495002,0.589378,0.552311,9.361949,0.842938,...,4,0.818441,0.083863,-0.242108,-0.014258,0.096396,0.417641,-0.050576,-0.204757,-0.172563
3,t_0f90acc7-d5c5-4e53-901d-55610fbd090c,237.653336,1988,96.796830,0.042606,0.389634,0.359044,0.585673,6.068578,0.665398,...,4,0.594829,0.192498,0.340039,0.034846,-0.389794,0.518381,0.185008,-0.079907,-0.016978
4,t_36b9ad02-095a-443d-a697-6c7285d9410a,174.600006,1987,97.905891,0.249982,0.513640,0.485435,0.635095,7.198735,0.408715,...,4,0.591289,0.270586,-0.411061,0.165898,0.225652,0.335518,-0.036643,-0.016300,-0.446870
